## Overview
The business has been investing in four different display advertising campaigns. You have been asked to evaluate how effective
each advertising campaign is in generating sales. Please see below the explanation of the data and the business questions.

The data on the left include information on 10000 consumers who clicked on at least one of the display ads from campaigns
A, B, C, or D. Purchase is indicated by the "Conversion" variable (i.e., equals 1 if there is purchase and 0 otherwise). The "Value" column indicates the revenue in dollars earned from each purchase. The cost per click is $7, $5, $4, and $2 for campaign A, B, C, and D, respectively. The order of clicks is as indicated in the data.

You are asked to build a statistical model (so not a rule based attribution model such as last click attribution) and answer:

Questions
1. Which campaign is the most successful in terms of unit sales contributed?

2. What is the return on investment for each campaign?

3. How would you optimize the spend of a given budget of $1 million across all four campaigns?

# Loading Data and Fill Missing WITH 'NULL'

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from pprint import pprint
from scipy.optimize import linprog

from helper import calculate_removal_effects, calculate_attribution, calculate_roi

df = pd.read_csv('./data/user_click.csv')  # Load your data into a pandas DataFrame
cost_per_click = {'a': 7, 'b': 5, 'c': 4, 'd': 2}  # Costs per click for each campaign
columns = ["user", "click_1", "click_2", "click_3", "click_4", "click_5", "conversion", "value"]
df = df[columns]
print('Data Overview')
display(df.head(5))
# Fill missing values with "END" placeholder
df[["click_1", "click_2", "click_3", "click_4", "click_5"]] = df[["click_1", "click_2", "click_3", "click_4", "click_5"]].fillna("NULL")

# Build sequence list: state 
df['sequence'] = df[['click_1', 'click_2', 'click_3', 'click_4', 'click_5', 'conversion']].values.tolist()
df['sequence'] = df['sequence'].apply(lambda x: [i for i in x if pd.notna(i)])

# Display the updated dataframe with the sequence column
print("Sequence and States")
display(df[['sequence', 'conversion', 'value']].head())

Data Overview


,user,click_1,click_2,click_3,click_4,click_5,conversion,value
0,1,d,d,a,b,NaN,0,0
1,2,c,NaN,NaN,NaN,NaN,0,0
2,3,b,d,a,b,c,1,44
3,4,b,a,a,NaN,NaN,0,0
4,5,d,b,d,NaN,NaN,0,0


Sequence and States


,sequence,conversion,value
0,"[d, d, a, b, NULL, 0]",0,0
1,"[c, NULL, NULL, NULL, NULL, 0]",0,0
2,"[b, d, a, b, c, 1]",1,44
3,"[b, a, a, NULL, NULL, 0]",0,0
4,"[d, b, d, NULL, NULL, 0]",0,0


#### The idea is to construct transition probability matrix between campaigns, to evaluate each campaign effect on conversion units

In [2]:
# Initialize dictionaries to count transitions and the total counts per state
transitions = defaultdict(int)
total_counts = defaultdict(int)

# Count transitions and total occurrences of each state
for seq in df['sequence']:
    for i in range(len(seq) - 1):
        transitions[(seq[i], seq[i + 1])] += 1
        total_counts[seq[i]] += 1
    total_counts[seq[-1]] += 1  # Last state occurrence

# Create the transition probability matrix
# Add 2 states: 0 = No conversion, 1 = Conversion
states = list(cost_per_click.keys())+[0]+[1]
transition_matrix = pd.DataFrame(np.zeros((len(states), len(states))), index=states, columns=states)

for (from_state, to_state), count in transitions.items():
    transition_matrix.loc[from_state, to_state] = count / total_counts[from_state]
transition_matrix = transition_matrix.fillna(0)
# Display the transition matrix
print("Transition Matrix")
pprint(transition_matrix)

Transition Matrix
             a         b         c         d         0         1      NULL
a     0.166486  0.174633  0.164584  0.164856  0.040060  0.024851  0.264530
b     0.165352  0.167850  0.164564  0.165484  0.039695  0.025237  0.271819
c     0.157222  0.169067  0.169067  0.165298  0.041728  0.022210  0.275407
d     0.165154  0.167457  0.164205  0.165696  0.044167  0.021271  0.272050
0     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
NULL  0.000000  0.000000  0.000000  0.000000  0.324661  0.074488  0.600851


# Removal Effects by Each Campaign

In [3]:
removal_effects = calculate_removal_effects(transition_matrix, cost_per_click)
print("Removal Effects of each Campaign")
pprint(removal_effects)

Removal Effects of each Campaign
{'a': -0.1373493815350687,
 'b': -0.13506083188630125,
 'c': -0.14303872488718505,
 'd': -0.1444904666185951}


# Conversion and Revenue Attribution

In [4]:
total_conversions = df['conversion'].sum()
total_revenue = df['value'].sum()
attribution_conversions, attribution_revenue = calculate_attribution(removal_effects, total_conversions, total_revenue)
print("Attribution (Conversions):")
pprint(attribution_conversions)
print("Attribution (Revenue or Value):")
pprint(attribution_revenue)

Attribution (Conversions):
{'a': 540.3811284921501,
 'b': 531.3771633632972,
 'c': 562.7650209176938,
 'd': 568.4766872268589}
Attribution (Revenue or Value):
{'a': 21569.62058684917,
 'b': 21210.22218937275,
 'c': 22463.086404619382,
 'd': 22691.070819158696}


In [5]:
total_click_by_campaign = {'a': 0, 'b': 0, 'c': 0, 'd': 0}
total_cost_by_campaign = {'a': 0, 'b': 0, 'c': 0, 'd': 0}
for i in range(1, 6):
    click_dict = df[f"click_{i}"].value_counts().to_dict()
    total_click_by_campaign['a'] += click_dict.get('a', 0)
    total_click_by_campaign['b'] += click_dict.get('b', 0)
    total_click_by_campaign['c'] += click_dict.get('c', 0)
    total_click_by_campaign['d'] += click_dict.get('d', 0)

for campaign, click in total_click_by_campaign.items():
    total_cost_by_campaign[campaign] = click*cost_per_click[campaign]
print("Total Click by Campaign")
display(total_click_by_campaign)
print("Total Cost by Campaign")
display(total_cost_by_campaign)


Total Click by Campaign


{'a': 7364, 'b': 7608, 'c': 7429, 'd': 7381}

Total Cost by Campaign


{'a': 51548, 'b': 38040, 'c': 29716, 'd': 14762}

# Return on Investment by each Campaign

In [6]:
roi = calculate_roi(attribution_revenue, total_cost_by_campaign)
print("Return on Investment by each campaign:") 
pprint({k: f"{round(100*v, 1)}%" for k,v in roi.items()})

Return on Investment by each campaign:
{'a': '-58.2%', 'b': '-44.2%', 'c': '-24.4%', 'd': '53.7%'}


# Budget Allocation Optimization with Linear Programming

In [7]:
# Coefficients for the objective function (negative because linprog does minimization)
c = [-roi[campaign] for campaign in total_cost_by_campaign.keys()]

# total removal effects 
re_sum = sum(removal_effects.values())

# Shorten the name of removal effects for convenience in writing matrix
re = removal_effects

# The equality constraints, assuming ROI of each campaign is constant:
# 1. Total budget = 1e6
# 2. Take into account removal effects
A_eq = [
    [(re['a']/re_sum - 1)*(roi['a']+1), re['a']*(roi['b'] + 1)/re_sum, re['a']*(roi['c'] + 1)/re_sum, re['a']*(roi['d'] + 1)/re_sum],
    [re['b']*(roi['a'] + 1)/re_sum, (re['b']/re_sum - 1)*(roi['b']+1), re['b']*(roi['c'] + 1)/re_sum, re['b']*(roi['d'] + 1)/re_sum],
    [re['c']*(roi['a'] + 1)/re_sum, re['c']*(roi['b'] + 1)/re_sum, (re['c']/re_sum - 1)*(roi['c']+1), re['c']*(roi['d'] + 1)/re_sum],
    [re['d']*(roi['a'] + 1)/re_sum, re['d']*(roi['b'] + 1)/re_sum, re['d']*(roi['c'] + 1)/re_sum, (re['d']/re_sum - 1)*(roi['d']+1)],
    [1, 1, 1, 1]
]
b_eq = [[0], [0], [0], [0], [1e6]]

budget=1e6

# Boundary of variables x should be non-pos
x_bounds = [(0, budget),
            (0, budget), 
            (0, budget),
            (0, budget)]

# Solve the linear programming problem
res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds)


# Extract the optimal budget allocation
budget_allocation = dict(zip(total_cost_by_campaign.keys(), res.x))

# Display the budget allocation
print("Estimated Budget Allocation by each campaign:")
pprint({k: f"{round(v)}" for k,v in budget_allocation.items()})
pprint({k: f"{round(100*v/budget)}%" for k,v in budget_allocation.items()})

Estimated Budget Allocation by each campaign:
{'a': '384497', 'b': '283741', 'c': '221652', 'd': '110110'}
{'a': '38%', 'b': '28%', 'c': '22%', 'd': '11%'}
